In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import mne
import re
import mne
import pandas as pd
import glob
from datetime import datetime
import os



In [2]:
def time_strings_to_seconds(time_strings):
    seconds_list = []

    for time_str in time_strings:
        try:
            time_obj = datetime.strptime(time_str, '%H:%M:%S')
        except ValueError:
            # If parsing with colon fails, try parsing with period
            try:
                time_obj = datetime.strptime(time_str, '%H:%M.%S')
            except ValueError:
            # If parsing with colon fails, try parsing with period
                time_obj = datetime.strptime(time_str, '%H.%M.%S')
        seconds = (time_obj.hour * 3600) + (time_obj.minute * 60) + time_obj.second
        seconds_list.append(seconds)
    return seconds_list


In [3]:
df = pd.read_csv('../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/subject_info.csv')
localization_column = df[' localization']
lateralization_column = df[' lateralization']
seizure_column = df[' seizure']


In [4]:
df

,patient_id,age_years,gender,seizure,localization,lateralization,eeg_channel,number_seizures,rec_time_minutes
0,PN00,55,Male,IAS,T,R,29,5,198
1,PN01,46,Male,IAS,T,L,29,2,809
2,PN03,54,Male,IAS,T,R,29,2,752
3,PN05,51,Female,IAS,T,L,29,3,359
4,PN06,36,Male,IAS,T,L,29,5,722
5,PN07,20,Female,IAS,T,L,29,1,523
6,PN09,27,Female,IAS,T,L,29,3,410
7,PN10,25,Male,FBTC,F,Bilateral,20,10,1002
8,PN11,58,Female,IAS,T,R,29,1,145
9,PN12,71,Male,IAS,T,L,29,4,246


In [5]:
ls ../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN11  #01, 10

index.html  PN11-1.edf  Seizures-list-PN11.txt


In [6]:
directory_list = list()
for root, dirs, files in os.walk("../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0", topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))

print(directory_list)

['../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN01', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN16', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN10', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN11', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN12', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN07', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN09', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN17', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN03', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN05', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN06', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN13', '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN14', '../shared_data/siena_ee

In [7]:
directory_list

['../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN01',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN16',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN10',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN11',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN12',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN07',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN09',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN17',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN03',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN05',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN06',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN13',
 '../shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN14',
 '../shared_

In [8]:
raws = []
labels = []
seizure_data = []
seizure_data_label = []
starts = []
ends = []

In [10]:
for i in range(4):   
    txt_files = glob.glob(directory_list[i]+"/*.txt")

    with open(txt_files[0], 'r') as file:
        content = file.read()

    #edf_files = glob.glob(directory_list[i]+"/*.edf")
    filenames = re.findall(r'File\s*name:\s*(\S+)', content)
    


    # Loop through each .edf file and read it
    for file_path in filenames:

        raw = mne.io.read_raw_edf(directory_list[i]+"/"+file_path)
        data = raw.to_data_frame()
        raws.append(data)
        labels.append(df.index[df['patient_id']==directory_list[i][-4:]][0])
        
    pattern = r'File\s*name:\s*(\S+)(?:\s*Registration\s*start\s*time:\s*([\d:.]+))?(?:\s*Registration\s*end\s*time:\s*([\d:.]+))?'

    # Initialize variables to store previous registration times
    prev_reg_start = None
    prev_reg_end = None

    # Initialize a list to store extracted data
    data = []

    # Iterate through the matches
    for match in re.finditer(pattern, content):
        file_name = match.group(1)
        reg_start = match.group(2)
        reg_end = match.group(3)

        # Use previous registration times if not provided for the current file
        if reg_start is None:
            reg_start = prev_reg_start
        else:
            prev_reg_start = reg_start

        if reg_end is None:
            reg_end = prev_reg_end
        else:
            prev_reg_end = reg_end

        data.append((file_name, reg_start, reg_end))

    registration_times_start = [entry[1] for entry in data]
    registration_times_end = [entry[2] for entry in data]

    seizure_times_start = re.findall(r'Seizure\s*start\s*time:\s*(\d{2}.\d{2}.\d{2})', content)
    seizure_times_end = re.findall(r'Seizure\s*end\s*time:\s*(\d{2}.\d{2}.\d{2})', content)
    seizure_times_start.extend(re.findall(r'Start\s*time:\s*(\d{2}.\d{2}.\d{2})', content))
    seizure_times_end.extend(re.findall(r'End\s*time:\s*(\d{2}.\d{2}.\d{2})', content))


    start = np.array(time_strings_to_seconds(seizure_times_start)) - np.array(time_strings_to_seconds(registration_times_start))
    end = np.array(time_strings_to_seconds(seizure_times_end)) - np.array(time_strings_to_seconds(registration_times_start))

    start = [i + 3600 * 24 if i < 0 else i for i in start]
    end = [i + 3600 * 24 if i < 0 else i for i in end]

    starts.extend(start)
    ends.extend(end)

    for j in range(labels.count(df.index[df['patient_id']==directory_list[i][-4:]][0])):
            seizure_data.append(raws[np.where(np.array(labels) == df.index[df['patient_id']==directory_list[i][-4:]][0])[0][j]].iloc[start[j]*512:end[j]*512].drop(['time', 'SPO2', '1', '2', 'MK','HR'], axis=1))
            seizure_data_label.append(i)
    

Extracting EDF parameters from /gpfs/accounts/eecs545w24_class_root/eecs545w24_class/shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN01/PN01-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /gpfs/accounts/eecs545w24_class_root/eecs545w24_class/shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN16/PN16-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /gpfs/accounts/eecs545w24_class_root/eecs545w24_class/shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN16/PN16-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /gpfs/accounts/eecs545w24_class_root/eecs545w24_class/shared_data/siena_eeg/physionet.org/files/siena-scalp-eeg/1.0.0/PN10/PN10-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure

In [11]:
seizure_data_df = pd.DataFrame(columns=['EEG', 'Seizure Type','Localization Type','Lateralization Type'], index=range(len(seizure_data)))
#seizure_data_df = pd.read_csv('seizure_data.csv')

for i in range(len(seizure_data)):
    seizure_data_df.loc[i]['EEG'] = seizure_data[i]
    seizure_data_df.loc[i]['Seizure Type'] = seizure_column[seizure_data_label[i]]
    seizure_data_df.loc[i]['Localization Type'] = localization_column[seizure_data_label[i]]
    seizure_data_df.loc[i]['Lateralization Type'] = lateralization_column[seizure_data_label[i]]
        

In [12]:
seizure_data_df.to_pickle('seizure_data.pkl')

In [13]:
seizure_data_df

,EEG,Seizure Type,Localization Type,Lateralization Type
0,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG ...,IAS,T,R
1,EEG Fp1 EEG F3 EEG C3 EEG P3 E...,IAS,T,L
2,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG...,IAS,T,L
3,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG ...,IAS,T,R
4,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG...,IAS,T,R
5,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG...,IAS,T,R
6,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG ...,IAS,T,R
7,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG ...,IAS,T,R
8,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG ...,IAS,T,R
9,EEG Fp1 EEG F3 EEG C3 EEG P3 EEG ...,IAS,T,R
